# Chapter 12: Custom Models and Training with TensorFlow

Notebook ini merupakan hasil reproduksi dan penjelasan teori dari **Bab 12 - Custom Models and Training with TensorFlow** dari buku *Hands-On Machine Learning with Scikit-Learn, Keras, and TensorFlow (2nd Edition)* oleh Aurélien Géron.

📌 Bab ini menjelaskan cara membuat model kustom menggunakan TensorFlow (beyond Keras Sequential/Functional API), termasuk subclassing model dan training loop manual.

---


## Ringkasan Teori Bab 12: Custom Models and Training with TensorFlow

### 1. Mengapa Perlu Model Kustom?

Walau Keras sangat fleksibel, terkadang kita perlu:
- Model kompleks dengan logika dinamis
- Kontrol penuh terhadap training loop
- Mengimplementasikan arsitektur eksperimental (misalnya GAN, Attention)

---

### 2. Subclassing Model

Alih-alih menggunakan `Sequential` atau `Functional API`, kita dapat membuat class kustom yang menurunkan `keras.Model`.

```python
class MyModel(keras.Model):
    def __init__(self):
        super().__init__()
        self.dense1 = keras.layers.Dense(10, activation="relu")
        self.out = keras.layers.Dense(1)

    def call(self, inputs):
        x = self.dense1(inputs)
        return self.out(x)
```

Keuntungan:
- Lebih fleksibel
- Bisa menambahkan loop, cabang, logika kondisi

---

### 3. Subclassing Layer

Mirip seperti subclassing model, kita juga bisa membuat custom layer dengan mewarisi `keras.layers.Layer`.

```python
class MyLayer(keras.layers.Layer):
    def __init__(self, units):
        super().__init__()
        self.units = units

    def build(self, input_shape):
        self.kernel = self.add_weight(...)

    def call(self, inputs):
        return tf.matmul(inputs, self.kernel)
```

---

### 4. Training Loop Manual

Dengan subclassed model, kita juga bisa menulis loop training secara manual.

Langkah umum:
1. Gunakan `tf.GradientTape()` untuk menghitung gradien
2. Terapkan optimizer
3. Update metrik
4. Loop sendiri per batch atau epoch

```python
with tf.GradientTape() as tape:
    y_pred = model(X_batch)
    loss = loss_fn(y_batch, y_pred)
grads = tape.gradient(loss, model.trainable_variables)
optimizer.apply_gradients(zip(grads, model.trainable_variables))
```

---

### 5. Custom Training Step dalam Model

Agar tetap bisa pakai `model.fit()`, kita bisa override `train_step()` dalam subclass model.

```python
class MyModel(keras.Model):
    def train_step(self, data):
        X, y = data
        with tf.GradientTape() as tape:
            y_pred = self(X, training=True)
            loss = self.compiled_loss(y, y_pred)
        grads = tape.gradient(loss, self.trainable_variables)
        self.optimizer.apply_gradients(zip(grads, self.trainable_variables))
        self.compiled_metrics.update_state(y, y_pred)
        return {m.name: m.result() for m in self.metrics}
```

---

### 6. Logging dan TensorBoard

Training loop manual bisa disambungkan ke TensorBoard dengan `tf.summary`.

---

### 7. Tips Penting

- `build()` hanya dipanggil sekali dan cocok untuk inisialisasi berat
- Gunakan `self.add_weight()` untuk manajemen parameter otomatis
- Selalu pastikan `call()` konsisten dengan mode `training=True/False`
- Hindari loop keras jika bisa ditulis sebagai layer komposit

---

Bab ini adalah pengantar penting menuju fleksibilitas penuh TensorFlow untuk eksperimen deep learning tingkat lanjut 🚀

---


## Implementasi Custom Model + Training Loop

Kita akan menggunakan dataset Iris dan membuat custom model `MyModel` dengan 2 dense layer, lalu melakukan training **secara manual** menggunakan `tf.GradientTape`.


In [ ]:
import tensorflow as tf
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.utils import to_categorical

# Load dan siapkan data
iris = load_iris()
X = iris.data
y = iris.target
y_cat = to_categorical(y)

X_train, X_test, y_train, y_test = train_test_split(X, y_cat, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train)).shuffle(100).batch(16)
test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(16)

In [ ]:
# Subclassed model
class MyModel(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.dense1 = tf.keras.layers.Dense(16, activation='relu')
        self.dense2 = tf.keras.layers.Dense(3, activation='softmax')

    def call(self, inputs):
        x = self.dense1(inputs)
        return self.dense2(x)

model = MyModel()

In [ ]:
# Optimizer dan loss function
loss_fn = tf.keras.losses.CategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()
train_acc_metric = tf.keras.metrics.CategoricalAccuracy()
test_acc_metric = tf.keras.metrics.CategoricalAccuracy()

In [ ]:
# Manual training loop
EPOCHS = 50
for epoch in range(EPOCHS):
    print(f"Epoch {epoch+1}/{EPOCHS}")
    for step, (x_batch, y_batch) in enumerate(train_ds):
        with tf.GradientTape() as tape:
            logits = model(x_batch, training=True)
            loss_value = loss_fn(y_batch, logits)
        grads = tape.gradient(loss_value, model.trainable_weights)
        optimizer.apply_gradients(zip(grads, model.trainable_weights))
        train_acc_metric.update_state(y_batch, logits)

    train_acc = train_acc_metric.result()
    print(f" - Train accuracy: {train_acc:.4f}")
    train_acc_metric.reset_state()

In [ ]:
# Evaluasi pada test set
for x_batch_test, y_batch_test in test_ds:
    test_logits = model(x_batch_test, training=False)
    test_acc_metric.update_state(y_batch_test, test_logits)

test_acc = test_acc_metric.result()
print(f"Test accuracy: {test_acc:.4f}")

### Penjelasan Kode:
- `MyModel` adalah subclass dari `tf.keras.Model` dengan dua layer.
- Training dilakukan **manual per batch** menggunakan `tf.GradientTape` untuk melacak dan menerapkan gradien.
- Kita tetap bisa pakai metrik Keras seperti `CategoricalAccuracy`.
- Metode ini memberi **kontrol penuh** atas training (misal: logika khusus, multiple optimizers, penyesuaian loss).

---
